In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.externals import joblib
from sklearn.ensemble import RandomForestClassifier
import gc
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score

SAMPLE_SIZE = 0.50
#Caso con edad en mean en vez de 200 para los nan unicamente
%matplotlib inline

In [2]:
avisos = pd.read_csv("Data/fiuba_entrenamiento/pocho/avisosONEHOT.csv")
postulaciones = pd.read_csv("Data/fiuba_entrenamiento/pocho/postulaciones.csv")
postulantes = pd.read_csv("Data/fiuba_entrenamiento/pocho/postulantesONEHOT.csv")

avisos['nombre_area']=avisos['nombre_area'].astype('category')
avisos_normales = pd.read_csv("Data/fiuba_entrenamiento/pocho/avisos.csv")
avisos_normales = avisos_normales.loc[:,["nombre_area","idaviso","tipo_de_trabajo","nivel_laboral"]]

vistas = pd.read_csv("Data/fiuba_entrenamiento/pocho/vistas.csv")

postulaciones['sepostulo']=1
postulaciones.drop(['fechapostulacion','dia','mes','semana','diadelasemana','hora'],axis=1,inplace=True)

In [3]:
vistas['cantidad']=0
vistas2=vistas.groupby(['idaviso','idpostulante'],as_index=False).agg({'cantidad':'count'})
vistas2.head()

,idaviso,idpostulante,cantidad
0,18,BolNL,1
1,48375,RwVdKR,1
2,169730,1KjXB,1
3,169730,2AKzxa,1
4,169730,6LJ64,1


In [4]:
vistas=vistas.groupby(['idpostulante'],as_index=False).agg({'cantidad':'count'});
vistas.head()

,idpostulante,cantidad
0,0002q,3
1,0005E,25
2,000R8,4
3,001XE,12
4,003k9,26


In [5]:
vistas3=pd.merge(vistas2,vistas,on='idpostulante',how='left')
vistas3.head()

,idaviso,idpostulante,cantidad_x,cantidad_y
0,18,BolNL,1,20
1,48375,RwVdKR,1,10
2,169730,1KjXB,1,34
3,169730,2AKzxa,1,1
4,169730,6LJ64,1,10


In [6]:
vistas3['cantidad']=vistas3['cantidad_x']/vistas3['cantidad_y']
vistas3.drop(['cantidad_x','cantidad_y'],axis=1,inplace=True)
vistas3.head()

,idaviso,idpostulante,cantidad
0,18,BolNL,0.050000
1,48375,RwVdKR,0.100000
2,169730,1KjXB,0.029412
3,169730,2AKzxa,1.000000
4,169730,6LJ64,0.100000


In [7]:
vistas=vistas.rename(columns={'cantidad': 'vistas_postulante'})
vistas2=vistas2.rename(columns={'cantidad': 'vistas_al_aviso'})
vistas_final = pd.merge(vistas3,avisos_normales, on= "idaviso", how = "left")
vistas_final.head()

,idaviso,idpostulante,cantidad,nombre_area,tipo_de_trabajo,nivel_laboral
0,18,BolNL,0.050000,NaN,NaN,NaN
1,48375,RwVdKR,0.100000,NaN,NaN,NaN
2,169730,1KjXB,0.029412,NaN,NaN,NaN
3,169730,2AKzxa,1.000000,NaN,NaN,NaN
4,169730,6LJ64,0.100000,NaN,NaN,NaN


In [8]:
no_postulaciones = pd.read_csv("Data/fiuba_entrenamiento/pocho/no-postulaciones/azar_total.csv")
postulaciones=postulaciones.append(no_postulaciones.sample(len(postulaciones)*3//4), ignore_index=True)
postulaciones.drop_duplicates(['idaviso','idpostulante'],keep='first',inplace=True)
postulaciones = postulaciones.sample(int(round(len(postulaciones)*SAMPLE_SIZE)))
no_postulaciones = 0
gc.collect()

C:\Users\Pocho\Anaconda3\lib\site-packages\pandas\core\frame.py:6201: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  sort=sort)


67

In [9]:
vistas_final["vistas_por_area"]=0
vistas_final["vistas_por_tipo"]=0
vistas_final["vistas_por_nivel"]=0

vistas_area = vistas_final.groupby(['idpostulante','nombre_area'],as_index=False).agg({'vistas_por_area':'count'})
vistas_tipo= vistas_final.groupby(['idpostulante','tipo_de_trabajo'],as_index=False).agg({'vistas_por_tipo':'count'})
vistas_nivel=vistas_final.groupby(['idpostulante','nivel_laboral'],as_index=False).agg({'vistas_por_nivel':'count'})

vistas_final = 0
gc.collect()

7

In [10]:
postulaciones.drop(['fechapostulacion','dia','mes','semana','diadelasemana','hora'],axis=1,inplace=True)

In [11]:
postulaciones=pd.merge(postulaciones,postulantes,on='idpostulante')
postulaciones=pd.merge(postulaciones,avisos,on='idaviso')
postulaciones = pd.merge(postulaciones,avisos_normales, on = "idaviso", how = "left")
postulaciones.drop("nombre_area_y", axis = 1, inplace=True)
postulaciones.rename({"nombre_area_x":"nombre_area"},axis=1,inplace=True)
postulaciones=pd.merge(postulaciones,vistas,on=['idpostulante'],how='left')
postulaciones=pd.merge(postulaciones,vistas2,on=['idaviso','idpostulante'],how='left')
postulaciones=pd.merge(postulaciones,vistas3,on=['idaviso','idpostulante'],how='left')
postulaciones=pd.merge(postulaciones,vistas_area,on=['idpostulante','nombre_area'],how='left')
postulaciones=pd.merge(postulaciones,vistas_tipo,on=['idpostulante','tipo_de_trabajo'],how='left')
postulaciones=pd.merge(postulaciones,vistas_nivel,on=['idpostulante','nivel_laboral'],how='left')
postulaciones['vistas_por_area']=postulaciones['vistas_por_area'].fillna(0)
postulaciones['vistas_por_tipo']=postulaciones['vistas_por_tipo'].fillna(0)
postulaciones['vistas_por_nivel']=postulaciones['vistas_por_tipo'].fillna(0)
postulaciones["vistas_al_aviso"] = postulaciones["vistas_al_aviso"].fillna(0)
postulaciones['lo_vio']=(postulaciones['vistas_al_aviso']>0)
postulaciones.cantidad = postulaciones.cantidad.fillna(0)
postulaciones.vistas_postulante = postulaciones.vistas_postulante.fillna(0)

postulaciones.sample(20)

,idaviso,idpostulante,sepostulo,edad,sexo_FEM,sexo_MASC,sexo_NO_DECLARA,Doctorado_Abandonado,Doctorado_En Curso,Doctorado_Graduado,...,nivel_laboral_Senior / Semi-Senior,tipo_de_trabajo,nivel_laboral,vistas_postulante,vistas_al_aviso,cantidad,vistas_por_area,vistas_por_tipo,vistas_por_nivel,lo_vio
1211687,1112428207,rm0OLbO,1,29.0,False,True,False,False,False,False,...,True,Full-time,Senior / Semi-Senior,118.0,1.0,0.008475,35.0,97.0,97.0,True
1877654,1112450445,jkxpw0B,1,26.0,False,True,False,False,False,False,...,True,Part-time,Senior / Semi-Senior,111.0,1.0,0.009009,24.0,4.0,4.0,True
1839473,1112367856,zv4Pzz5,1,22.0,False,True,False,False,False,False,...,True,Full-time,Senior / Semi-Senior,65.0,1.0,0.015385,15.0,55.0,55.0,True
627704,1112303815,QNr4dOq,1,34.0,True,False,False,False,False,False,...,True,Full-time,Senior / Semi-Senior,1.0,0.0,0.000000,1.0,1.0,1.0,False
5189189,1112427984,NzrOAAB,0,20.0,True,False,False,False,False,False,...,True,Full-time,Senior / Semi-Senior,1.0,0.0,0.000000,0.0,1.0,1.0,False
1917071,1112448433,6rQja5M,0,27.0,False,True,False,False,False,False,...,True,Full-time,Senior / Semi-Senior,0.0,0.0,0.000000,0.0,0.0,0.0,False
19344,1112278786,wVk42Gw,1,32.0,True,False,False,False,False,False,...,True,Full-time,Senior / Semi-Senior,0.0,0.0,0.000000,0.0,0.0,0.0,False
2097767,1112363471,0zPakYv,0,37.0,True,False,False,False,False,False,...,False,Full-time,Junior,9.0,0.0,0.000000,0.0,6.0,6.0,False
460928,1112280067,Nzj03rB,1,33.0,True,False,False,False,False,False,...,True,Full-time,Senior / Semi-Senior,317.0,0.0,0.000000,12.0,273.0,273.0,False
1854048,1112406827,1krzO,1,46.0,False,True,False,False,False,False,...,False,Full-time,Gerencia / Alta Gerencia / Dirección,6.0,1.0,0.166667,1.0,6.0,6.0,True


In [12]:
postulaciones_area = postulaciones.groupby(['idpostulante','nombre_area'],as_index=False).agg({'sepostulo':'count'})
postulaciones_tipo= postulaciones.groupby(['idpostulante','tipo_de_trabajo'],as_index=False).agg({'sepostulo':'count'})
postulaciones_nivel =postulaciones.groupby(['idpostulante','nivel_laboral'],as_index=False).agg({'sepostulo':'count'})
postulaciones_area=postulaciones_area.rename(columns={'sepostulo':'postulaciones_misma_area'})
postulaciones_tipo=postulaciones_tipo.rename(columns={'sepostulo':'postulaciones_mismo_tipo'})
postulaciones_nivel=postulaciones_nivel.rename(columns={'sepostulo':'postulaciones_mismo_nivel'})

In [13]:
postulaciones.dropna(axis = 1, inplace=True)

In [14]:
postulaciones_area.head()

,idpostulante,nombre_area,postulaciones_misma_area
0,0z5Dmrd,Call Center,1
1,0z5Dmrd,Ingeniería Mecánica,1
2,0z5Dmrd,Recursos Humanos,2
3,0z5Dmrd,Selección,1
4,0z5Dmrd,Tecnologia / Sistemas,1


In [15]:
postulaciones=pd.merge(postulaciones,postulaciones_area,on=['idpostulante','nombre_area'],how='left')
postulaciones=pd.merge(postulaciones,postulaciones_tipo,on=['idpostulante','tipo_de_trabajo'],how='left')
postulaciones=pd.merge(postulaciones,postulaciones_nivel,on=['idpostulante','nivel_laboral'],how='left')

postulaciones['postulaciones_misma_area']=postulaciones['postulaciones_misma_area'].fillna(0).apply(lambda x: x if(x<=0) else x-1)
postulaciones['postulaciones_mismo_nivel']=postulaciones['postulaciones_mismo_nivel'].fillna(0).apply(lambda x: x if(x<=0) else x-1)
postulaciones['postulaciones_mismo_tipo']=postulaciones['postulaciones_mismo_tipo'].fillna(0).apply(lambda x: x if(x<=0) else x-1)
postulaciones.drop(["tipo_de_trabajo","nivel_laboral"], axis = 1, inplace=True)



In [16]:
#mean encoding
area_mean = postulaciones.groupby("nombre_area").agg({"sepostulo":"mean"})
area_mean.head()

means = area_mean.to_dict()["sepostulo"]
postulaciones.nombre_area = postulaciones.nombre_area.replace(means)
postulaciones.sample(10)

,idaviso,idpostulante,sepostulo,edad,sexo_FEM,sexo_MASC,sexo_NO_DECLARA,Doctorado_Abandonado,Doctorado_En Curso,Doctorado_Graduado,...,vistas_postulante,vistas_al_aviso,cantidad,vistas_por_area,vistas_por_tipo,vistas_por_nivel,lo_vio,postulaciones_misma_area,postulaciones_mismo_tipo,postulaciones_mismo_nivel
1257825,1112432492,5mL1P4X,0,31.000000,True,False,False,False,False,False,...,1.0,0.0,0.000000,0.0,1.0,1.0,False,0,3,2
4751251,1112405843,1QrNRjR,0,30.000000,True,False,False,False,False,False,...,2.0,0.0,0.000000,0.0,0.0,0.0,False,0,0,0
1300868,1112323181,vVewp6G,1,33.000000,False,True,False,False,False,False,...,1.0,0.0,0.000000,0.0,1.0,1.0,False,1,11,5
430628,1112322670,Yj5peXq,0,30.777591,False,False,True,False,False,False,...,0.0,0.0,0.000000,0.0,0.0,0.0,False,1,3,3
5197859,1112452839,A3vqd5Y,1,33.000000,False,True,False,False,False,False,...,147.0,1.0,0.006803,17.0,111.0,111.0,True,7,90,24
2715051,1111786150,6al15R,0,51.000000,True,False,False,False,False,False,...,3.0,0.0,0.000000,1.0,2.0,2.0,False,4,14,3
5525604,1111985910,0zdQllr,0,30.777591,False,False,True,False,False,False,...,2.0,0.0,0.000000,0.0,1.0,1.0,False,0,4,0
5013788,1112228384,QNLlWrV,0,26.000000,True,False,False,False,False,False,...,5.0,0.0,0.000000,0.0,2.0,2.0,False,0,3,4
1975241,1112283554,6r2DqZM,1,25.000000,False,True,False,False,False,False,...,0.0,0.0,0.000000,0.0,0.0,0.0,False,5,23,22
134094,1112331701,vVjOwNx,0,30.777591,False,False,True,False,False,False,...,1.0,0.0,0.000000,0.0,0.0,0.0,False,0,5,3


In [17]:
train, test=train_test_split(postulaciones,test_size=0.10)
print("Train: ",len(train),"Test: ",len(test))
features=list(postulaciones.columns)
features.remove('idaviso')
features.remove('idpostulante')
features.remove('sepostulo')
features.remove('titulo')
features.remove('denominacion_empresa')
features.remove("ciudad")
features.remove("mapacalle")

x_train=train[features]
y_train=train['sepostulo']

x_test=test[features]
y_test=test['sepostulo']

postulaciones=0

gc.collect()

Train:  5011526 Test:  556837


98

In [18]:
model = XGBClassifier(objective='binary:logistic',max_depth=18,learning_rate=0.1,subsample=0.8,colsample_bytree=0.8,n_estimators=1200,silent=False,n_jobs=-1,verbose=1)
dt=model.fit(x_train,y_train,eval_set=[(x_test,y_test)],verbose=True,early_stopping_rounds=5)

[0]	validation_0-error:0.11225
Will train until validation_0-error hasn't improved in 5 rounds.
[1]	validation_0-error:0.107549
[2]	validation_0-error:0.105706
[3]	validation_0-error:0.105027
[4]	validation_0-error:0.103885
[5]	validation_0-error:0.10247
[6]	validation_0-error:0.102314
[7]	validation_0-error:0.101505
[8]	validation_0-error:0.099821
[9]	validation_0-error:0.09869
[10]	validation_0-error:0.098316
[11]	validation_0-error:0.097242
[12]	validation_0-error:0.097116
[13]	validation_0-error:0.096533
[14]	validation_0-error:0.096183
[15]	validation_0-error:0.095755
[16]	validation_0-error:0.095358
[17]	validation_0-error:0.09483
[18]	validation_0-error:0.094543
[19]	validation_0-error:0.094214
[20]	validation_0-error:0.093821
[21]	validation_0-error:0.093685
[22]	validation_0-error:0.093349
[23]	validation_0-error:0.093164
[24]	validation_0-error:0.092681
[25]	validation_0-error:0.092359
[26]	validation_0-error:0.092004
[27]	validation_0-error:0.09184
[28]	validation_0-error:0.

In [19]:
from sklearn.metrics import roc_auc_score
x_final=x_test[features]
y_final=model.predict_proba(x_final)
y_pred=[]
for elem in y_final:
    y_pred.append(elem[1])
score=roc_auc_score(y_test, y_pred)
print(score)

0.9818936267168297


In [20]:
print("dale mono")

dale mono


In [21]:
joblib.dump(model, "Data/fiuba_entrenamiento/pocho/modelos/XGBoostMean4.pkl")

['Data/fiuba_entrenamiento/pocho/modelos/XGBoostMean4.pkl']

In [22]:
prediccion = pd.read_csv("Data/fiuba_entrenamiento/test_final_100k.csv")
prediccion = pd.merge(prediccion,postulantes,on='idpostulante',how='inner')
prediccion = pd.merge(prediccion,avisos,on='idaviso',how='inner')
prediccion = pd.merge(prediccion,avisos_normales,on='idaviso',how='inner')
prediccion.drop("nombre_area_y", axis = 1, inplace=True)
prediccion.rename({"nombre_area_x":"nombre_area"},axis=1,inplace=True)

prediccion = pd.merge(prediccion,vistas,on='idpostulante',how='left')
prediccion.sample(10)


prediccion=pd.merge(prediccion,vistas2,on=['idaviso','idpostulante'],how='left')
prediccion=pd.merge(prediccion,vistas3,on=['idaviso','idpostulante'],how='left')
prediccion['cantidad']=prediccion['cantidad'].fillna(0)
prediccion['vistas_postulante']=prediccion['vistas_postulante'].fillna(0)
prediccion['vistas_al_aviso']=prediccion['vistas_al_aviso'].fillna(0)
prediccion['lo_vio']=(prediccion['vistas_al_aviso']>0)

prediccion=pd.merge(prediccion,postulaciones_area,on=['idpostulante','nombre_area'],how='left')
prediccion=pd.merge(prediccion,postulaciones_tipo,on=['idpostulante','tipo_de_trabajo'],how='left')
prediccion=pd.merge(prediccion,postulaciones_nivel,on=['idpostulante','nivel_laboral'],how='left')
prediccion=pd.merge(prediccion,vistas_area,on=['idpostulante','nombre_area'],how='left')
prediccion=pd.merge(prediccion,vistas_tipo,on=['idpostulante','tipo_de_trabajo'],how='left')
prediccion=pd.merge(prediccion,vistas_nivel,on=['idpostulante','nivel_laboral'],how='left')
prediccion['vistas_por_area']=prediccion['vistas_por_area'].fillna(0)
prediccion['vistas_por_tipo']=prediccion['vistas_por_tipo'].fillna(0)
prediccion['vistas_por_nivel']=prediccion['vistas_por_tipo'].fillna(0)
prediccion['postulaciones_misma_area']=prediccion['postulaciones_misma_area'].fillna(0)
prediccion['postulaciones_mismo_nivel']=prediccion['postulaciones_mismo_nivel'].fillna(0)
prediccion['postulaciones_mismo_tipo']=prediccion['postulaciones_mismo_tipo'].fillna(0)
prediccion.nombre_area = prediccion.nombre_area.replace(means)
prediccion.sample(10)

,id,idaviso,idpostulante,edad,sexo_FEM,sexo_MASC,sexo_NO_DECLARA,Doctorado_Abandonado,Doctorado_En Curso,Doctorado_Graduado,...,vistas_postulante,vistas_al_aviso,cantidad,lo_vio,postulaciones_misma_area,postulaciones_mismo_tipo,postulaciones_mismo_nivel,vistas_por_area,vistas_por_tipo,vistas_por_nivel
24183,79198,1112464639,6rZrlxO,28.0,False,True,False,False,False,False,...,14.0,0.0,0.000000,False,0.0,39.0,20.0,0.0,12.0,12.0
4289,65348,1112458667,jkdaXXO,35.0,False,True,False,False,False,False,...,106.0,1.0,0.009434,True,0.0,0.0,12.0,4.0,2.0,2.0
95865,95361,1112488545,1vjRab,38.0,False,True,False,False,False,False,...,1.0,0.0,0.000000,False,0.0,8.0,5.0,0.0,1.0,1.0
99760,99730,1112513517,EbQ2z8,28.0,False,True,False,False,False,False,...,21.0,0.0,0.000000,False,0.0,7.0,0.0,0.0,17.0,17.0
28760,67134,1112460162,1PakPR,39.0,True,False,False,False,False,False,...,4.0,0.0,0.000000,False,0.0,7.0,5.0,0.0,3.0,3.0
78712,26051,1112336485,52OLXZ,47.0,True,False,False,False,False,False,...,7.0,0.0,0.000000,False,0.0,5.0,3.0,0.0,0.0,0.0
34577,31214,1112383996,RzPl6Vq,35.0,False,True,False,False,False,False,...,0.0,0.0,0.000000,False,2.0,7.0,6.0,0.0,0.0,0.0
4940,472,1803860,eVevN2,47.0,True,False,False,False,False,False,...,25.0,0.0,0.000000,False,0.0,23.0,5.0,0.0,11.0,11.0
74885,53473,1112445172,wVYzoPQ,27.0,False,True,False,False,False,False,...,18.0,1.0,0.055556,True,3.0,10.0,7.0,6.0,17.0,17.0
97895,97697,1112502341,Z0a0zb,42.0,False,True,False,False,False,False,...,3.0,0.0,0.000000,False,1.0,6.0,5.0,0.0,3.0,3.0


In [23]:
prediccion.head()

,id,idaviso,idpostulante,edad,sexo_FEM,sexo_MASC,sexo_NO_DECLARA,Doctorado_Abandonado,Doctorado_En Curso,Doctorado_Graduado,...,vistas_postulante,vistas_al_aviso,cantidad,lo_vio,postulaciones_misma_area,postulaciones_mismo_tipo,postulaciones_mismo_nivel,vistas_por_area,vistas_por_tipo,vistas_por_nivel
0,0,739260,6M9ZQR,42.0,True,False,False,False,False,True,...,26.0,0.0,0.0,False,0.0,3.0,0.0,3.0,20.0,20.0
1,1,739260,6v1xdL,31.0,False,True,False,False,False,False,...,4.0,0.0,0.0,False,0.0,23.0,0.0,0.0,3.0,3.0
2,2,739260,ezRKm9,36.0,True,False,False,False,False,False,...,10.0,0.0,0.0,False,0.0,9.0,1.0,1.0,6.0,6.0
3,3,758580,1Q35ej,68.0,False,True,False,False,False,False,...,0.0,0.0,0.0,False,0.0,5.0,0.0,0.0,0.0,0.0
4,4,758580,EAN4J6,32.0,True,False,False,False,False,False,...,1.0,0.0,0.0,False,1.0,4.0,0.0,0.0,1.0,1.0


In [24]:
x_final=prediccion[features]
y_final=model.predict_proba(x_final)
y_final2=[]
print(model.classes_)
print(len(y_final))
for elem in y_final:
    y_final2.append(elem[1])
print(len(y_final2))

prediccion['sepostulo']=y_final2
prediccion.head()


[0 1]
100000
100000


,id,idaviso,idpostulante,edad,sexo_FEM,sexo_MASC,sexo_NO_DECLARA,Doctorado_Abandonado,Doctorado_En Curso,Doctorado_Graduado,...,vistas_al_aviso,cantidad,lo_vio,postulaciones_misma_area,postulaciones_mismo_tipo,postulaciones_mismo_nivel,vistas_por_area,vistas_por_tipo,vistas_por_nivel,sepostulo
0,0,739260,6M9ZQR,42.0,True,False,False,False,False,True,...,0.0,0.0,False,0.0,3.0,0.0,3.0,20.0,20.0,0.034333
1,1,739260,6v1xdL,31.0,False,True,False,False,False,False,...,0.0,0.0,False,0.0,23.0,0.0,0.0,3.0,3.0,0.020628
2,2,739260,ezRKm9,36.0,True,False,False,False,False,False,...,0.0,0.0,False,0.0,9.0,1.0,1.0,6.0,6.0,0.053190
3,3,758580,1Q35ej,68.0,False,True,False,False,False,False,...,0.0,0.0,False,0.0,5.0,0.0,0.0,0.0,0.0,0.002041
4,4,758580,EAN4J6,32.0,True,False,False,False,False,False,...,0.0,0.0,False,1.0,4.0,0.0,0.0,1.0,1.0,0.003128


In [25]:
prediccion=prediccion[['id','sepostulo']]
print(prediccion.sepostulo.mean())
prediccion.head()

0.4951478307780821


,id,sepostulo
0,0,0.034333
1,1,0.020628
2,2,0.053190
3,3,0.002041
4,4,0.003128


In [26]:
prediccion.to_csv("Data/fiuba_entrenamiento/pocho/predicciones/xgbmean4.csv",index=False)